# SQLite Database Integration Tutorial

This tutorial demonstrates the SQLite database integration for the Lexos `corpus` module. The database integration provides:

- **Full-text search**: Efficient search across document content in the corpus
- **Metadata queries**: Filter and aggregate corpus statistics
- **Flexible deployment options**: In-memory, file-based, or hybrid storage
- **Data Integrity**: Hash verification and transaction safety

In [ ]:
# Import the database-enabled corpus classes
from lexos.corpus.sqlite import create_corpus

print("Database integration loaded successfully!")

## Create a Basic Database-Enabled Corpus

Create a corpus with database integration enabled alongside file storage:

In [ ]:
# Create a database-enabled corpus with dual storage
corpus = create_corpus(
    corpus_dir="demo_corpus",
    sqlite_path="demo_corpus.db",
    name="Demo Corpus",
    sqlite_only=False  # Use both files and database
)

print(f"Created corpus: {corpus.name}")
print(f"Database enabled: {corpus.use_sqlite}")
print(f"Database path: {corpus.db.database_path if corpus.db else 'None'}")

Once you have created a database-enabled corpus, all the corpus attributes, properties, and methods of the `Corpus` class are available for your use.

## Adding Documents

Add documents that will be stored in both the file system and database:

In [ ]:
# Sample documents for demonstration
documents = [
    {
        "content": "The quick brown fox jumps over the lazy dog. This pangram contains every letter of the alphabet.",
        "name": "pangram_sample",
        "metadata": {"genre": "example", "language": "english", "type": "pangram"}
    },
    {
        "content": "Machine learning is a powerful tool for analyzing large datasets and extracting meaningful patterns.",
        "name": "ml_description",
        "metadata": {"genre": "technical", "language": "english", "type": "description"}
    },
    {
        "content": "Shakespeare wrote many famous plays including Hamlet, Romeo and Juliet, and Macbeth during the Elizabethan era.",
        "name": "shakespeare_info",
        "metadata": {"genre": "literature", "language": "english", "type": "biographical"}
    },
    {
        "content": "Climate change represents one of the most significant challenges facing humanity in the 21st century.",
        "name": "climate_statement",
        "metadata": {"genre": "science", "language": "english", "type": "statement"}
    }
]

# Add documents to the corpus
for doc in documents:
    corpus.add(
        content=doc["content"],
        name=doc["name"],
        metadata=doc["metadata"]
    )

print(f"Added {len(documents)} documents to corpus")
print(f"Total records: {corpus.num_docs}")
print(f"Active records: {corpus.num_active_docs}")

## Use Full-Text Search Capabilities

Use the `search()` method to leverage SQLite's FTS5 full-text search functionality:

In [ ]:
# Search for documents containing specific terms
search_queries = [
    "machine learning",
    "Shakespeare",
    "climate change",
    "fox jumps",
    "alphabet OR patterns"
]

for query in search_queries:
    results = corpus.search(query, limit=10)
    print(f"\nSearch: '{query}'")
    print(f"Found {len(results)} results:")

    for record in results:
        print(f"  - {record.name}: {record.preview}")

You can perform more complex searches using SQL syntax or wildcards:

In [ ]:
# Complex boolean search
complex_query = '("machine learning" OR "artificial intelligence") AND dataset*'
complex_results = corpus.search(complex_query)
print(f"Complex query results: {len(complex_results)}")

# Prefix search
prefix_results = corpus.search('Shakespear*')
print(f"Prefix search results: {len(prefix_results)}")

## Advanced Filtering

You can use database queries to filter records by various criteria. For this, use the `filter_records()` method. Parameters are:

- `is_active`: A boolean indicating whether the record is active
- `is_parsed`: A boolean indicating whether the record is active
- `model`: A string indicating the record's language model
- `min_tokens`: An integer indicating the minimum number of tokens in the record's content
- `max_tokens`: An integer indicating the maximum number of tokens in the record's content
- `limit`: An integer indicating the maximum number of results to return

In [ ]:
# Filter by token count range
short_docs = corpus.filter_records(min_tokens=1, max_tokens=15)
print(f"Short documents (1-15 tokens): {len(short_docs)}")
for doc in short_docs:
    print(f"  - {doc.name}: {doc.num_tokens() if doc.is_parsed else 'not parsed'} tokens")

# Filter by active status
active_docs = corpus.filter_records(is_active=True)
print(f"\nActive documents: {len(active_docs)}")

# Filter by parsing status
parsed_docs = corpus.filter_records(is_parsed=True)
print(f"Parsed documents: {len(parsed_docs)}")

## Database Statistics

You can get comprehensive statistics directly from the database:

In [ ]:
# Get database-derived statistics
db_stats = corpus.get_stats()

print("Database Statistics:")
print(f"  Total records: {db_stats['total_records']}")
print(f"  Active records: {db_stats['active_records']}")
print(f"  Parsed records: {db_stats['parsed_records']}")
print(f"  Total tokens: {db_stats['total_tokens']}")
print(f"  Total terms: {db_stats['total_terms']}")
print(f"  Average vocab density: {db_stats['average_vocab_density']:.2f}")

## Database-Only Mode

If you want to store corpus records only in the database, rather than in separate files, you can create a corpus that uses only database storage by setting `sqlite_only=True`:

In [ ]:
# Create database-only corpus
db_only_corpus = create_corpus(
    corpus_dir="memory_corpus",
    sqlite_path=":memory:",  # In-memory database
    name="Memory Corpus",
    sqlite_only=True  # No file storage
)

# Add some documents
test_docs = [
    "This is a test document for the memory-based corpus.",
    "Another example showing database-only storage capabilities.",
    "Fast in-memory operations for temporary analysis workflows."
]

for i, content in enumerate(test_docs):
    db_only_corpus.add(
        content=content,
        name=f"memory_doc_{i+1}",
        metadata={"source": "memory_test", "doc_id": i+1}
    )

print(f"Database-only corpus created with {db_only_corpus.num_docs} documents")

# Search in memory corpus
memory_results = db_only_corpus.search("memory OR operations")
print(f"Search results in memory corpus: {len(memory_results)}")
for result in memory_results:
    print(f"  - {result.name}")

## Data Synchronization

You can also synchronize records between file-based and database storage:

In [ ]:
from lexos.corpus import Corpus
from lexos.corpus.sqlite import SQLiteCorpus

# Create a traditional file-based corpus first
file_corpus = Corpus(corpus_dir="sync_test_corpus", name="File Corpus")

# Add some documents to file corpus
sync_docs = [
    "Document one for synchronization testing.",
    "Document two with different content for sync.",
    "Final document to test the sync process."
]

for i, content in enumerate(sync_docs):
    file_corpus.add(
        content=content,
        name=f"sync_doc_{i+1}",
        metadata={"sync_test": True, "file_id": i+1}
    )

print(f"Created file corpus with {file_corpus.num_docs} documents")

# Now create database-enabled corpus and sync from files
sync_corpus = SQLiteCorpus(
    corpus_dir="sync_test_corpus",  # Same directory
    sqlite_path="sync_test.db",
    name="File Corpus",  # Same name
    use_sqlite=True
)

# Load existing file-based records
# sync_corpus.load(corpus_dir="sync_test_corpus")

# Synchronize to database
synced_count = sync_corpus.sync()
print(f"Synchronized {synced_count} records to database")

# Test search on synchronized data
sync_results = sync_corpus.search("synchronization OR sync")
print(f"Search results in synchronized corpus: {len(sync_results)}")

## Cleanup

If you have been using this notebook to test the functionality of the SQLite integration, now is a good time to delete the database and record files you created. Run this cell to perform a clean up.

In [ ]:
import shutil
import os

# List of test directories and files to clean up
cleanup_items = [
    "demo_corpus",
    "demo_corpus.db",
    "sync_test_corpus",
    "sync_test.db",
    "memory_corpus"
]

print("Cleaning up test files and directories:")
for item in cleanup_items:
    try:
        if os.path.isdir(item):
            shutil.rmtree(item)
            print(f"  - Removed directory: {item}")
        elif os.path.isfile(item):
            os.remove(item)
            print(f"  - Removed file: {item}")
    except Exception as e:
        print(f"  - Could not remove {item}: {str(e)}")

print("\nCleanup completed!")